# RecSys
Modelo do tipo recuperação

## Importando bibliotecas necessárias

In [2]:
from sklearn import model_selection, metrics, preprocessing
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import torch.nn as nn
import pandas as pd
import numpy as np
import torch

## Carregando dados do dataset de avaliações de filmes

In [3]:
df_olist_txn = pd.read_csv('data/olist/rfzd_olist_transactions.csv')
df_olist_txn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100965 entries, 0 to 100964
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   customer_id               100965 non-null  object
 1   product_id                100965 non-null  object
 2   customer_zip_code_prefix  100965 non-null  int64 
 3   product_category_name     100965 non-null  object
 4   purchase_count            100965 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 3.9+ MB


In [4]:
df_olist_txn.sample(5)

,customer_id,product_id,customer_zip_code_prefix,product_category_name,purchase_count
71727,0d4b78e7e121e21dbdf23cf165fea19b,7a10781637204d8d10485c71a6108a2e,88085,relogios_presentes,1
69028,ad7e368f155a2a9da1d1599cb84827d0,cc158c284564106f4b668ae02389e2c3,13570,telefonia,1
68184,ffc25a124da207c0b90a002ebfaa7184,fd939dd54384f5681c327919e8f6a1b8,13660,ferramentas_jardim,1
17650,4faf7bb3041b910f0a590121c4480f8c,681e1525258578dd8a22a3c249509bf5,86020,malas_acessorios,1
95750,6a19aac11f8b7b0541420cfd0b4bd3a5,4ae1bd4115a0ea08489ff7e821ddc4e5,20541,moveis_decoracao,1


Checando distribuição de compras (de 1 a 20) em porcentagem

In [7]:
df_olist_txn['purchase_count'].value_counts(normalize=True) * 100

purchase_count
1     93.078790
2      5.256277
3      0.930025
4      0.380330
6      0.168375
5      0.163423
10     0.004952
7      0.003962
9      0.001981
8      0.001981
20     0.001981
12     0.001981
15     0.001981
14     0.001981
13     0.000990
11     0.000990
Name: proportion, dtype: float64

## Preprocessamento dos dados para a geração de embeddings

Remapeando IDs para que fiquem sequênciais e incrementais

In [23]:
labels_customer = preprocessing.LabelEncoder()
labels_products = preprocessing.LabelEncoder()
labels_zip_code = preprocessing.LabelEncoder()
labels_category = preprocessing.LabelEncoder()

In [24]:
df_olist_txn['customer_id'] = labels_customer.fit_transform(df_olist_txn['customer_id'].values)
df_olist_txn['product_id'] = labels_products.fit_transform(df_olist_txn['product_id'].values)
df_olist_txn['customer_zip_code_prefix'] = labels_zip_code.fit_transform(df_olist_txn['customer_zip_code_prefix'].values)
df_olist_txn['product_category_name'] = labels_category.fit_transform(df_olist_txn['product_category_name'].values)


In [25]:
df_olist_txn.sample(5)

,customer_id,product_id,customer_zip_code_prefix,product_category_name,purchase_count
38738,92894,15855,2153,26,1
87238,96912,21724,14498,54,2
24245,62331,30304,391,63,1
14047,27438,28635,3084,70,1
15081,6859,21130,11326,66,1


Dividindo dataset entre conjunto de treinamento e teste

In [26]:
df_train, df_test = model_selection.train_test_split(
    df_olist_txn,
    test_size=0.2,
    random_state=12
)

Configurando dataset para o treinamento do modelo

In [27]:
class OlistDataset:
    '''
    Classe criada com o intuito de ajustar o dataset pandas ao
    treinamento de modelos utilizando o PyTorch, especialmente do que se
    diz respeito à utilização de lotes (batches) durante o treinamento.
    '''
    def __init__(self, customers, products, purchases):
        self.customers = customers
        self.products = products
        self.purchases = purchases

    def __len__(self):
        return (len(self.customers))
    
    def __getitem__(self, item):

        customers = self.customers[item]
        products = self.products[item]
        purchases = self.purchases[item]

        return {
            "user": torch.tensor(customers, dtype=torch.long),
            "movies": torch.tensor(products, dtype=torch.long),
            "ratings": torch.tensor(purchases, dtype=torch.long),
        }

Criando datasets de treinamento e teste

In [28]:
train_dataset = OlistDataset(
    customers=df_train['customer_id'].values,
    products=df_train['product_id'].values,
    purchases=df_train['purchase_count'].values
)

test_dataset = OlistDataset(
    customers=df_test['customer_id'].values,
    products=df_test['product_id'].values,
    purchases=df_test['purchase_count'].values
)

In [29]:
train_dataset[:10]

{'user': tensor([91914, 89644, 54576, 57461, 94005, 80559, 55628, 81226, 36837, 42585]),
 'movies': tensor([30394, 23495, 20790, 21139, 29531,  1022,   505, 29287, 25834, 31003]),
 'ratings': tensor([3, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

Ajustando Data para ser utilizado pelo PyTorch

In [47]:
batch_size = 32

In [48]:
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

Após isso, toda vez que o DataLoader for requisitado, ele retornará um lote (batch) de 32 itens

## Criando modelo de RecSys

Criando modelo de RecSys no estilo de torre-dupla

In [49]:
class OlistRecSys(nn.Module):
    '''
    Classe criada com o intuito de modelar a estrutura de torre-dupla,
    isto é, um dos modelos clássicos de RecSys baseado em filtragem
    colaborativa por meio de redes neurais.
    '''
    def __init__(self, n_customers, n_products, embedding_size = 32):
        super().__init__()
        # definindo embedding para clientes e produtos
        self.customer_embedding = nn.Embedding(n_customers, embedding_size)
        self.products_embedding = nn.Embedding(n_products, embedding_size)
        # definindo camada de saída como um neurônio
        self.output_layer = nn.Linear(embedding_size * 2, 1)

    def forward(self, customers, products, purchases = None):
        # criando camada de entrada a partir de embeddings de clientes e produtos
        customer_embeddings = self.customer_embedding(customers)
        products_embeddings = self.products_embedding(products)
        # concatenando embeddings de clientes e produtos
        concat_embeddings = torch.cat([customer_embeddings, products_embeddings], dim=1)
        output = self.output_layer(concat_embeddings)
        return output

Configurando dispositivo para utilizar GPU se possível; caso contrário, CPU

In [50]:
mode = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(mode)
device

device(type='cuda')

Contando quantidade clientes e produtos distintos envolvidos em compras

In [51]:
n_customers = len(labels_customer.classes_)
n_products = len(labels_products.classes_)
n_customers, n_products

(97277, 32341)

Instanciando modelo RecSys, configurando otimizador, taxa de aprendizado e função custo

In [53]:
model = OlistRecSys(n_customers, n_products).to(device)

optimizer = torch.optim.Adam(model.parameters())
scheaduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.7)
loss_function = nn.MSELoss()

Criando o ciclo de treinamento

In [54]:
epochs = 1
total_loss = 0
plot_steps = 5000
print_steps = 5000
step_count = 0
all_losses_list = []

In [ ]:
model.train()
for epoch_index in range(epochs):
    for index, train_data in enumerate(train_loader):
        # predições do modelo (y-predito)
        output = model(train_data['customers'], train_data['products'])
        # reformatando y-verdeiro para fical igual ao formato da saída do modelo y predito
        purchases = train_data['purchases'].view(batch_size, -1).to(torch.float32)
        # calculando o erro do modelo
        loss = loss_function(output, purchases)
        # somando erro do modelo ao longo dos passos de treinamento
        total_loss += loss.sum().item()
        # executando ajuste dos pesos no modelo via algoritmo de retropropagação
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # contando quantidade de instâncias utilizadas a cada iteração do treinamento
        step_count += batch_size

        if (step_count % plot_steps == 0):
            average_loss = total_loss / ( batch_size * plot_steps)
            print(f'epoch {epoch_index} loss at step: {step_count} is {average_loss}')
            all_losses_list.append(average_loss)
            total_loss = 0

In [ ]:
plt.figure()
plt.plot(all_losses_list)
plt.show()

## Avaliando o modelo

In [ ]:
from sklearn.metrics import mean_squared_error

model_output_list = []
target_purchasing_list = []

model.eval()

with torch.no_grad():

    for index, test_data in enumerate(test_loader):
        output = model(test_data['customers'], test_data['products'])

        model_output_list.append(output.sum().item() / batch_size)